In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ./drive/MyDrive/최애림/cakd7/m9_딥러닝/3차 프로젝트/5팀(윤정준, 최애림, 진청아, 이정현a)/코드

/content/drive/.shortcut-targets-by-id/1niQkF2oFYkuV2aahMfHbpa3kXs5yrcAd/5팀(윤정준, 최애림, 진청아, 이정현a)/코드


# 필요 모듈 임포트

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

# TEST

### 패치데이터 불러오기

In [ ]:
train_patches = np.load('./array/1125_real_train_1_5(5).npy', allow_pickle= True)
train_patches

### 모델 불러오기

In [ ]:
# 종양(종양 및 비종양 균형(인덱싱) 모델) 예측모델
model = load_model('./Checkpoints/InceptionV3-Freeze40-UnbalData/1124_tumor_best_model.hdf5')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_transpose_3 (Conv2DT  (None, 96, 96, 3)        81        
 ranspose)                                                       
                                                                 
 inception_v3 (Functional)   (None, 1, 1, 2048)        21802784  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 21,804,914
Trainable params: 2,130
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
train_patches.shape

(100,)

In [ ]:
train_patches[0].shape

(15252, 32, 32, 3)

In [ ]:
train_patches[0][0].shape

(32, 32, 3)

In [ ]:
for i in range(2):
  print(f'*** 이미지{i+1}번째')
  invasive = 0
  non_invasive = 0
  for patch in train_patches[i]:
    patch = (patch.astype('float32') / 255.) * 2.0 - 1
    preds = model.predict(patch[None,:], verbose = 0)
    if preds > 0.5:
      invasive += 1
    else:
      non_invasive += 1
    
  print(f'invasive 개수: {invasive}')
  print(f'non_invasive 개수: {non_invasive}')

*** 이미지1번째
invasive 개수: 3268
non_invasive 개수: 11984
*** 이미지2번째
invasive 개수: 3336
non_invasive 개수: 16194


In [ ]:
3268/(3268 + 11984), 3336/(3336 + 16194)

(0.2142669813794912, 0.17081413210445467)

# 모델 예측 후 배열 저장하기

In [ ]:
181 * 86

15566

In [ ]:
np.arange(181 *  86).reshape(181,86)

array([[    0,     1,     2, ...,    83,    84,    85],
       [   86,    87,    88, ...,   169,   170,   171],
       [  172,   173,   174, ...,   255,   256,   257],
       ...,
       [15308, 15309, 15310, ..., 15391, 15392, 15393],
       [15394, 15395, 15396, ..., 15477, 15478, 15479],
       [15480, 15481, 15482, ..., 15563, 15564, 15565]])

In [ ]:
train_size

array([[124, 123],
       [186, 105],
       [197,  78],
       [197,  87],
       [176,  95],
       [202,  81],
       [165,  86],
       [191,  72],
       [181,  71],
       [186,  98],
       [191,  85],
       [191,  93],
       [191,  85],
       [176, 107],
       [181,  91],
       [191,  90],
       [155,  89],
       [186,  99],
       [181,  79],
       [171,  90],
       [176,  90],
       [171, 104],
       [186,  82],
       [181, 104],
       [165,  87],
       [165,  71],
       [181,  86],
       [108,  79],
       [145, 123],
       [124, 114],
       [171, 106],
       [197,  92],
       [160,  71],
       [165,  97],
       [155,  87],
       [191,  96],
       [171,  85],
       [160,  90],
       [186,  90],
       [197,  60],
       [181,  74],
       [176,  76],
       [186, 103],
       [181, 101],
       [186,  93],
       [ 88,  53],
       [181,  86],
       [181,  89],
       [155,  87],
       [186,  94],
       [186,  92],
       [176,  80],
       [165,

In [ ]:
# 원본 사진 형태대로 저장하기 위해 사이즈 배열파일 불러오기
train_size = np.load('./array/1123_real_train_size_200_300(100).npy', allow_pickle= True)

# 전체 마스킹된 하나의 이미지를 담을 빈 리스트 생성 
all_mask_label = []

# 100개 이미지에 대해 마스킹
for i in range(100):
  print(f'*** 이미지{i+1}번째')
  # 이미지 한 개당 사이즈 측정
  size_col, size_row = train_size[i]

  # 출력을 위한 종양, 비종양 개수 객체 만들기
  tumor = 0
  non_tumor = 0

  # 하나의 이미지에 해당하는 패치들의 마스킹 라벨 담을 빈 리스트 생성
  mask_label = []

  # 이미지 한 개당 여러 패치 마스킹
  for patch in train_patches[i]:

    # 정규화
    patch = (patch.astype('float32') / 255.) * 2.0 - 1
    
    # 학습된 모델로 예측
    preds = model.predict(patch[None,:], verbose = 0)
    ## 예측값이 0.5보다 크면 종양
    if preds > 0.5:
      tumor += 1
      mask_label.append(1)

    ## 예측값이 0.5보다 작으면 비종양
    else:
      non_tumor += 1
      mask_label.append(0)
  
  # 이미지 사이즈에 맞게 배열 조정하고 all_mask_label리스트에 담기
  all_mask_label.append(np.array(mask_label).reshape(size_row, size_col))

  print(f'tumor 개수: {tumor}')
  print(f'non_tumor 개수: {non_tumor}')
  all_mask_label_arr = np.array(all_mask_label)

  # 종양 및 비종양 개수 맞춘 모델 적용 후 저장
  np.save('./array/1125_tumor_predict_애림.npy', all_mask_label_arr)